In [1]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 0C52-665E

 Directory of C:\Users\lic\Documents\PythonScripts\BostonMarathon

03/02/2017  11:31 AM    <DIR>          .
03/02/2017  11:31 AM    <DIR>          ..
03/02/2017  10:07 AM    <DIR>          .ipynb_checkpoints
03/02/2017  11:31 AM             6,387 BMwithsplits.ipynb
03/02/2017  11:27 AM           165,452 bm_split2014.csv
03/02/2017  10:06 AM            33,017 BostonMarathon.ipynb
03/02/2017  09:59 AM    <DIR>          Data
03/02/2017  09:59 AM                63 README.md
               4 File(s)        204,919 bytes
               4 Dir(s)  28,770,603,008 bytes free


In [2]:
import requests
from bs4 import BeautifulSoup
import os.path
import pandas as pd
import re
import string
BAAsite = 'http://registration.baa.org/'
url_suffix = '/cf/Public/iframe_ResultsSearch.cfm?mode=results'
columns=['bib','name','age','gender','city','state', 'country','ctz',
         't5k','t10k','t15k', 't20k','half','t25k','t30k','t35k','t40k',
         'pace','projt','offt','overcall','gdiv','div']

#2014 ma
#gender 0, 1: M, 2: F,

In [3]:
def get_with_year_lastname_gender(year,lastname,gender=0,start=1):
    param = {'StoredProcParamsOn': 'yes',
                 'LastName': lastname,
                 'GenderID': gender,
                 'VarTargetCount': 1000,
                 'records':25,
                 'start': start,
                 "RaceYearLowID": str(year),
                 'next': 'Next+25+Records'}
    url=BAAsite+str(year)+url_suffix
    r = requests.post(url, param)
    return r

def save_splitdata_tocsv(df_bm,year):
    filename=''.join(['bm_split',str(year),'.csv'])
    if os.path.isfile(filename):
        with open(filename,'a') as f:
            df_bm.to_csv(f,index = False,header=False,encoding='utf-8')
    else:
        df_bm.to_csv(filename,index = False,header=True,encoding='utf-8')
        
def bm_split_list(request_result):
    df_bm=pd.DataFrame(columns=columns)
    soup = BeautifulSoup(request_result.text,"lxml")
    maintable=soup.find_all(name='tr', attrs={'class': 'tr_header'})
    subtable = soup.find_all(name="table", attrs={"class": "table_infogrid"})
    num=len(maintable)
    if num>0:
        for i in range(num):
            (bib,name,age,gender,city,state,country,ctz,_)=[t.text.strip() for t in maintable[i].findAll("td")]
            (t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div )= [t.text.strip() for t in subtable[i].findAll("td")]
            df_bm.loc[i]=[bib,name,age,gender,city,state,country,ctz,t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div]
    return df_bm, num

In [4]:
string.ascii_lowercase  #python 3? 

'abcdefghijklmnopqrstuvwxyz'

In [7]:
def test(year=2014, lastname='ma'):
    for gender in [1,2]:
        for start in list(range(1,1000,25)):
            #print('{},').format(start),
            print(start),
            requested=get_with_year_lastname_gender(year,lastname,gender,start)
            df_bm,num=bm_split_list(requested)
            if num==0: 
                break;
            save_splitdata_tocsv(df_bm,year)
                

In [8]:
test(2014,'li')

1
26
51
76
101
126
1
26
51
76
101


In [9]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 0C52-665E

 Directory of C:\Users\lic\Documents\PythonScripts\BostonMarathon

03/02/2017  11:33 AM    <DIR>          .
03/02/2017  11:33 AM    <DIR>          ..
03/02/2017  10:07 AM    <DIR>          .ipynb_checkpoints
03/02/2017  11:33 AM             5,788 BMwithsplits.ipynb
03/02/2017  11:32 AM           196,133 bm_split2014.csv
03/02/2017  10:06 AM            33,017 BostonMarathon.ipynb
03/02/2017  09:59 AM    <DIR>          Data
03/02/2017  09:59 AM                63 README.md
               4 File(s)        235,001 bytes
               4 Dir(s)  28,769,288,192 bytes free


In [ ]:
!git add 